In [1]:
import os
import cv2
from PIL import Image
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import auc, roc_curve
from tqdm import tqdm

import time

C:\Users\aquat\AppData\Local\Temp\ipykernel_7532\236138535.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
import keras
import tensorflow as tf
from keras import layers
from keras.models import Model
from keras.layers import Lambda, concatenate
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D , Conv2D , MaxPooling2D
from keras.layers import  Dropout , BatchNormalization , Dense
from keras.optimizers import Adam
from keras.applications import ResNet50
from keras.callbacks import Callback , ReduceLROnPlateau , ModelCheckpoint
from sklearn.metrics import cohen_kappa_score, accuracy_score
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy

In [3]:
# Loading Dataset

nonDemented_train = np.load('/CSE499A/Train_Test/Non_Demented_Train.npy')
nonDemented_test = np.load('/CSE499A/Train_Test/Non_Demented_Test.npy')

veryMildDemented_train = np.load('/CSE499A/Train_Test/VeryMild_Demented_Train.npy')
veryMildDemented_test = np.load('/CSE499A/Train_Test/VeryMild_Demented_Test.npy')

mildDemented_train = np.load('/CSE499A/Train_Test/Mild_Demented_Train.npy')
mildDemented_test = np.load('/CSE499A/Train_Test/Mild_Demented_Test.npy')

moderateDemented_train = np.load('/CSE499A/Train_Test/Moderate_Demented_Train.npy')
moderateDemented_test = np.load('/CSE499A/Train_Test/Moderate_Demented_Test.npy')


# Check if the datasets are loaded

if nonDemented_train.size > 0 and veryMildDemented_train.size > 0 and mildDemented_train.size > 0 and moderateDemented_train.size > 0:
    print('Dataset Loaded!')
else:
    print('Dataset Load failed!')

Dataset Loaded!


In [4]:
# Shape of our dataset
print(f'Non Demented Train: ',nonDemented_train.shape)
print(f'Very Mild Demented Train: ',veryMildDemented_train.shape)
print(f'Mild Demented Train: ',mildDemented_train.shape)
print(f'Moderate Demented Train: ',moderateDemented_train.shape)
print(f'Non Demented Test: ',nonDemented_test.shape)
print(f'Very Mild Demented Test: ',veryMildDemented_test.shape)
print(f'Mild Demented Test: ',mildDemented_test.shape)
print(f'Moderate Demented Test: ',moderateDemented_test.shape)

Non Demented Train:  (2400, 128, 128, 3)
Very Mild Demented Train:  (1680, 128, 128, 3)
Mild Demented Train:  (672, 128, 128, 3)
Moderate Demented Train:  (48, 128, 128, 3)
Non Demented Test:  (800, 128, 128, 3)
Very Mild Demented Test:  (560, 128, 128, 3)
Mild Demented Test:  (224, 128, 128, 3)
Moderate Demented Test:  (16, 128, 128, 3)


In [5]:
# Set to class 0 and class 3
nonDemented_train_label = np.zeros(len(nonDemented_train),dtype=float)
veryMildDemented_train_label = np.zeros(len(veryMildDemented_train),dtype=float)
mildDemented_train_label = np.ones(len(mildDemented_train),dtype=float)
moderateDemented_train_label = np.ones(len(moderateDemented_train),dtype=float)

nonDemented_test_label = np.zeros(len(nonDemented_test),dtype=float)
veryMildDemented_test_label = np.zeros(len(veryMildDemented_test),dtype=float)
mildDemented_test_label = np.ones(len(mildDemented_test),dtype=float)
moderateDemented_test_label = np.ones(len(moderateDemented_test),dtype=float)

X_train = np.concatenate((nonDemented_train, veryMildDemented_train, mildDemented_train, moderateDemented_train), axis = 0)
Y_train = np.concatenate((nonDemented_train_label, veryMildDemented_train_label, mildDemented_train_label, moderateDemented_train_label), axis = 0)
X_test = np.concatenate((nonDemented_test, veryMildDemented_test, mildDemented_test, moderateDemented_test), axis = 0)
Y_test = np.concatenate((nonDemented_test_label, veryMildDemented_test_label, mildDemented_test_label, moderateDemented_test_label), axis = 0)

s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]

Y_train = to_categorical(Y_train, num_classes= 2)
Y_test = to_categorical(Y_test, num_classes= 2)

In [6]:
#printing the shape
print(f'X train shape:',X_train.shape)
print(f'X test shape:',X_test.shape)
print(f'Y train shape:',Y_train.shape)
print(f'Y test shape:',Y_test.shape)

X train shape: (4800, 128, 128, 3)
X test shape: (1600, 128, 128, 3)
Y train shape: (4800, 2)
Y test shape: (1600, 2)


In [7]:
#Spliting

x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=10)

In [8]:
print(f'x train shape:',x_train.shape)
print(f'x val shape:',x_val.shape)
print(f'y train shape:',y_train.shape)
print(f'y val shape:',y_val.shape)

x train shape: (3840, 128, 128, 3)
x val shape: (960, 128, 128, 3)
y train shape: (3840, 2)
y val shape: (960, 2)


In [11]:
from keras.applications import VGG16

def vgg16_model(backbone , lr = 1e-4):
  modelvgg16 = Sequential()
  modelvgg16.add(backbone)
  modelvgg16.add(layers.GlobalAveragePooling2D())
  modelvgg16.add(layers.Dropout(0.5))
  modelvgg16.add(layers.BatchNormalization())
  modelvgg16.add(layers.Dense(2 , activation='softmax'))
  
  modelvgg16.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=lr),
        metrics=['accuracy']
    )
    
  return modelvgg16


vgg16 = VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = (128 , 128 , 3)
)
# call the model 
modelvgg16 = vgg16_model(vgg16 , lr = 1e-4)
modelvgg16.build((None, 224, 224, 3))
modelvgg16.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14,717,762
Trainable params: 14,716,738


In [12]:
# Train the student model with MobileNet
epochs = 10
batch_size = 64
student_his_vgg16 = modelvgg16.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val))

Epoch 1/10
60/60 [==============================] - 630s 11s/step - loss: 0.7530 - accuracy: 0.5539 - val_loss: 0.4055 - val_accuracy: 0.8562
Epoch 2/10
60/60 [==============================] - 626s 10s/step - loss: 0.6382 - accuracy: 0.6378 - val_loss: 0.4021 - val_accuracy: 0.8562
Epoch 3/10
60/60 [==============================] - 622s 10s/step - loss: 0.5466 - accuracy: 0.7448 - val_loss: 0.6644 - val_accuracy: 0.6167
Epoch 4/10
60/60 [==============================] - 750s 13s/step - loss: 0.5158 - accuracy: 0.8078 - val_loss: 8.2693 - val_accuracy: 0.1437
Epoch 5/10
60/60 [==============================] - 699s 12s/step - loss: 0.4809 - accuracy: 0.8391 - val_loss: 0.4860 - val_accuracy: 0.8562
Epoch 6/10
60/60 [==============================] - 706s 12s/step - loss: 0.4545 - accuracy: 0.8484 - val_loss: 0.4161 - val_accuracy: 0.8562
Epoch 7/10
60/60 [==============================] - 738s 12s/step - loss: 0.4394 - accuracy: 0.8484 - val_loss: 0.4123 - val_accuracy: 0.8562
Epoch 

In [13]:
Y_val_pred = modelvgg16.predict(x_val)
print(f'The VGG16 Accuracy on the Validation Set:',accuracy_score(np.argmax(y_val, axis=1), np.argmax(Y_val_pred, axis=1)))

30/30 [==============================] - 40s 1s/step
The VGG16 Accuracy on the Validation Set: 0.85625


In [14]:
#Now let's check my Y_test values
print(f'My Y_test values are:\n' ,Y_test)

My Y_test values are:
 [[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [16]:
#Now let's check my predcited values from X_test dataset	
import timeit

start = timeit.default_timer()
#Your statements here
y_pred = modelvgg16.predict(X_test)
print(f'My predicted Y_test values are:\n' ,y_pred)

stop = timeit.default_timer()
print('\nTime: ',stop - start,'sec')

50/50 [==============================] - 67s 1s/step
My predicted Y_test values are:
 [[0.9978606  0.00213934]
 [0.9967794  0.00322061]
 [0.9969626  0.00303733]
 ...
 [0.9971687  0.00283123]
 [0.9970796  0.00292039]
 [0.9974101  0.00258984]]

Time:  67.365269300004 sec


In [17]:
#Now let's check the accuracy between the original & predicted (Y_test , y_pred)
print(f'My accuracy on Student model with VGG16 on the Test set is:',accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_pred, axis=1)))

My accuracy on Student model with VGG16 on the Test set is: 0.85


In [18]:
# Print the classification report
print(classification_report( np.argmax(Y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92      1360
           1       0.00      0.00      0.00       240

    accuracy                           0.85      1600
   macro avg       0.42      0.50      0.46      1600
weighted avg       0.72      0.85      0.78      1600



c:\Users\aquat\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\aquat\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\aquat\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,